In [2]:
from math import inf as infinity
from random import choice
import platform
import time
import json
from os import system

HUMAN = -1
COMP = +1
game_board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]

parent_child = {}
visited_list = []
terminals = []
Scores = {}
levels = {}
child_parent = {}
child_parent['---------'] = [None]

In [3]:
def stateGenerator(intial_state, pos, player):
    state = ""
    for i, row in enumerate(intial_state):
        if row != 'X' and row != 'O' and i == pos:
            state += player
        else:
            state += row
    if state == intial_state:
        return None
    return state
def Terminal_State(State):
    if State[0] == State[1] == State[2] =='O':
        return(True,-1)
    elif State[3] == State[4] == State[5] == 'O':
        return(True,-1)
    elif State[6] == State[7] == State[8] =='O':
        return(True,-1)
    elif State[0] == State[3] == State[6] =='O':
        return(True,-1)
    elif State[1] == State[4] == State[7] =='O':
        return(True,-1)
    elif State[2] == State[5] == State[8] =='O':
        return(True,-1)
    elif State[0] == State[4] == State[8] =='O':
        return(True,-1)
    elif State[2] == State[4] == State[6] =='O':
        return(True,-1)
    if State[0] == State[1] == State[2] =='X':
        return(True,1)
    elif State[3] == State[4] == State[5] == 'X':
        return(True,1)
    elif State[6] == State[7] == State[8] =='X':
        return(True,1)
    elif State[0] == State[3] == State[6] =='X':
        return(True,1)
    elif State[1] == State[4] == State[7] =='X':
        return(True,1)
    elif State[2] == State[5] == State[8] =='X':
        return(True,1)
    elif State[0] == State[4] == State[8] =='X':
        return(True,1)
    elif State[2] == State[4] == State[6] =='X':
        return(True,1)

    for i in State:
        if i =='-':
            return(False,0)
    return(True,0)
def isTerminal(state):
    return Terminal_State(state)[0]
def winning_score(state):
    return Terminal_State(state)[1]

In [4]:
def Tree(node):
    fringe = []
    fringe.append(node)
    while len(fringe) != 0:
        p = fringe.pop(0)
        yield p
        if (p.count('X') + p.count('O')) % 2 == 0:
            player = 'X'
            Scores[p] = [winning_score(p), 'O']
        else:
            player = 'O'
            Scores[p] = [winning_score(p), 'X']
        if not isTerminal(p):
            childs = generate_childs(p, player)
            parent_child[p] = childs
            for c in childs:
                fringe.append(c)
        else:
            if p not in terminals:
                terminals.append(p)
            parent_child[p] = childs

def generate_childs(node, player):
    children = []
    if player == 'X':
        for j, row in enumerate(node):
            state = stateGenerator(node, j, player)
            if state is not None:
                children.append(state)
                if state not in child_parent.keys():
                    child_parent[state] = []
                child_parent[state].append(node)
    else:
        for j, row in enumerate(node):
            state = stateGenerator(node, j, player)
            if state is not None:
                children.append(state)
                if state not in child_parent.keys():
                    child_parent[state] = []
                child_parent[state].append(node)

    return children


In [5]:
Scores['---------']=[0, 'O']
start_node = '---------'
k = list(Tree(start_node))
import json
with open('Tree.json', 'w+') as f:
    json.dump(parent_child, f)

with open('child_parent.json', 'w+') as f:
    json.dump(child_parent, f)


In [6]:
def evaluate_board(state):
    if is_winner(state, COMP):
        score = +1
    elif is_winner(state, HUMAN):
        score = -1
    else:
        score = 0

    return score

def is_winner(state, player):
    patterns = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    return [player, player, player] in patterns

def is_game_over(state):
    return is_winner(state, HUMAN) or is_winner(state, COMP)

def find_empty_cells(state):
    cells = []
    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])
    return cells

def is_valid_move(x, y, state):
    return [x, y] in find_empty_cells(state)

def make_move(x, y, player, state):
    if is_valid_move(x, y, state):
        state[x][y] = player
        return True
    else:
        return False


In [7]:
def minimax(state, depth, player):
    if player == COMP:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or is_game_over(state):
        score = evaluate_board(state)
        return [-1, -1, score]

    for cell in find_empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best

In [8]:
def clear_screen():
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')

def display_board(state, comp_choice, human_choice):
    symbols = {
        -1: human_choice,
        +1: comp_choice,
        0: ' '
    }
    line = '---------------'

    print('\n' + line)
    for row in state:
        for cell in row:
            symbol = symbols[cell]
            print(f'| {symbol} |', end='')
        print('\n' + line)

def computer_turn(comp_choice, human_choice):
    depth = len(find_empty_cells(game_board))
    if depth == 0 or is_game_over(game_board):
        return

    clear_screen()
    print(f'Computer turn [{comp_choice}]')
    display_board(game_board, comp_choice, human_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(game_board, depth, COMP)
        x, y = move[0], move[1]

    make_move(x, y, COMP, game_board)
    time.sleep(1)

def human_turn(comp_choice, human_choice):
    depth = len(find_empty_cells(game_board))
    if depth == 0 or is_game_over(game_board):
        return

    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clear_screen()
    print(f'Human turn [{human_choice}]')
    display_board(game_board, comp_choice, human_choice)

    while move < 1 or move > 9:
        try:
            move = int(input('Use numpad (1..9): '))
            coord = moves[move]
            can_move = make_move(coord[0], coord[1], HUMAN, game_board)

            if not can_move:
                print('Bad move')
                move = -1
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

def main():
    clear_screen()
    human_choice = ''
    comp_choice = ''
    first = ''

    while human_choice != 'O' and human_choice != 'X':
        try:
            print('')
            human_choice = input('Choose X or O\nChosen: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    if human_choice == 'X':
        comp_choice = 'O'
    else:
        comp_choice = 'X'

    clear_screen()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    while len(find_empty_cells(game_board)) > 0 and not is_game_over(game_board):
        if first == 'N':
            computer_turn(comp_choice, human_choice)
            first = ''

        human_turn(comp_choice, human_choice)
        computer_turn(comp_choice, human_choice)

    if is_winner(game_board, HUMAN):
        clear_screen()
        print(f'Human turn [{human_choice}]')
        display_board(game_board, comp_choice, human_choice)
        print('YOU WIN!')
    elif is_winner(game_board, COMP):
        clear_screen()
        print(f'Computer turn [{comp_choice}]')
        display_board(game_board, comp_choice, human_choice)
        print('YOU LOSE!')
    else:
        clear_screen()
        display_board(game_board, comp_choice, human_choice)
        print('DRAW!')

    with open('Tree.json', 'w+') as f:
        json.dump(parent_child, f)

    with open('child_parent.json', 'w+') as f:
        json.dump(child_parent, f)

    exit()

In [9]:
main()


Choose X or O
Chosen: x
First to start?[y/n]: y
Human turn [X]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 1
Computer turn [O]

---------------
| X ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
| X ||   ||   |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 2
Computer turn [O]

---------------
| X || X ||   |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
| X || X || O |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 7
Computer turn [O]

---------------
| X || X || O |
---------------
|   || O ||   |
---------------
| X ||   ||   |
---------------
Human turn [X]

---------------
| X || X || O |
---------------
| O || O ||   |
---------------
| X ||   ||   |